In [1]:
import numpy as np
import pandas as pd 
import torch
from torch import nn
time = np.loadtxt("../Data/AAL/100408_timeseries.txt")

In [2]:
time.shape

(1200, 116)

In [3]:
from Dataset import TimeSDataset
Dataset = TimeSDataset("../Data/AAL/","Gender",label_path="../Data/Behavioral-HCP.csv")

In [4]:
data , label = next(iter(Dataset))
data = torch.stack([data,data])
data.shape , label

(torch.Size([2, 1200, 116]), tensor(1))

In [5]:
label.shape

torch.Size([])

In [9]:
from Config import Config
from SplitWindows import SplitWindows
import torch
SplitedSeries = SplitWindows(data)

In [10]:
SplitedSeries.shape   # ( B , T , P , V )

torch.Size([2, 11, 20, 116])

In [11]:
Config.T

11

In [12]:
from AttentionArea import TemporalAttention , SpatialAttention
TAttention = TemporalAttention (8 ,Config.V * Config.K_E , 32 )

In [13]:
Temporal_output , weights = TAttention(SplitedSeries)
# (B , T , K_E , V)
Temporal_output.shape , weights.shape

(torch.Size([2, 11, 40, 116]), torch.Size([2, 11, 11]))

In [14]:
# Spatial_output = SAttention(Temporal_output)

In [15]:
Config.V * Config.K_E 


4640

In [23]:
class DynamicMatrix(nn.Module):
    def __init__(self):
        super().__init__()
        # initial Query and Key Parameters
        self.W_Q = nn.Parameter(torch.rand(Config.K_E , Config.K_S))
        self.W_K = nn.Parameter(torch.rand(Config.K_E , Config.K_S))
        self.Sigmoid = nn.Sigmoid()
    def forward(self , x):
        # INPUT : (B , T , K_E , V)
        x = x.view(x.size(0), x.size(1),x.size(3),x.size(2)) # Reshape to : (B , T , V , K_E)
        Q_T = x @ self.W_Q  # (B , T , V , K_S)
        K_T = x @ self.W_K  # (B , T , V , K_S)
        return self.Sigmoid((Q_T @ K_T.transpose(2,3)) / torch.sqrt(torch.tensor(Config.K_S)))

DM = DynamicMatrix()
weights = DM(Temporal_output)
weights.shape

torch.Size([2, 11, 116, 116])